In [9]:
import open3d as o3d
import numpy as np
from sklearn.cluster import DBSCAN

# Load your lidar point cloud data (replace 'lidar_data.pcd' with your actual .pcd file)
lidar_cloud = o3d.io.read_point_cloud('IDD_3D/20220118103808_seq_0/lidar/00000.pcd')

# Convert the point cloud to a NumPy array
lidar_points = np.asarray(lidar_cloud.points)

# Preprocess the point cloud (e.g., remove ground points, downsample)
# For simplicity, we assume you've already preprocessed the data

# Object detection (DBSCAN clustering)
dbscan = DBSCAN(eps=0.5, min_samples=10)
labels = dbscan.fit_predict(lidar_points)

# Extract unique labels to identify clusters
unique_labels = np.unique(labels)

# Object localization (calculating centroids)
object_positions = []
for label in unique_labels:
    if label == -1:
        continue  # Skip noise points labeled as -1
    cluster_points = lidar_points[labels == label]
    centroid = np.mean(cluster_points, axis=0)
    object_positions.append(centroid)

# Visualize the localized objects
visualized_objects = [o3d.geometry.PointCloud()]
for pos in object_positions:
    object_cloud = o3d.geometry.PointCloud()
    object_cloud.points = o3d.utility.Vector3dVector(np.array([pos]))
    object_cloud.paint_uniform_color([1, 0, 0])  # Color the object points in red
    visualized_objects.append(object_cloud)

o3d.visualization.draw_geometries([lidar_cloud] + visualized_objects)


In [10]:
import open3d as o3d
import numpy as np

# Load the LiDAR point cloud data
pcd = o3d.io.read_point_cloud("IDD_3D/20220118103808_seq_0/lidar/00000.pcd")

# Load 3D bounding box labels from the JSON file
# Replace 'labels.json' with the path to your JSON file
import json
with open('IDD_3D/20220118103808_seq_0/label/00000.json', 'r') as f:
    labels = json.load(f)

# Create a dictionary to map object types to colors for visualization
class_to_color = {
    "MotorcycleRider": [1.0, 0.0, 0.0],  # Red color for MotorcycleRider
    "Pedestrian": [1.0, 0.0, 0.0],
    "Car": [0.0, 1.0, 0.0],
    "Scooter": [0.0, 0.0, 1.0],
    "TourCar": [0.0, 1.0, 0.0],
    "Truck": [0.0, 1.0, 0.0]# Add more object types and corresponding colors as needed
}

# Visualize the LiDAR point cloud with bounding boxes
o3d.visualization.draw_geometries([pcd])

# Extract and visualize objects from the point cloud
for label in labels:
    obj_type = label["obj_type"]
    obj_id = label["obj_id"]
    position = label["psr"]["position"]
    rotation = label["psr"]["rotation"]
    scale = label["psr"]["scale"]

    # Create a bounding box for visualization
    color = class_to_color.get(obj_type, [0.0, 1.0, 0.0])  # Default to green if not in the dictionary
    bbox = o3d.geometry.OrientedBoundingBox()
    bbox.center = np.array([position["x"], position["y"], position["z"]])
    bbox.R = np.array([[np.cos(rotation["z"]), -np.sin(rotation["z"]), 0],
                      [np.sin(rotation["z"]), np.cos(rotation["z"]), 0],
                      [0, 0, 1]])
    bbox.extent = np.array([scale["x"], scale["y"], scale["z"]])

    # Visualize the bounding box in the point cloud
    o3d.visualization.draw_geometries([pcd, bbox])

# Additional processing and object detection logic can be added here



[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.


In [11]:
import open3d.ml.torch as ml3d
import open3d.ml.torch.models
import open3d.ml.torch.pipelines
import open3d.ml.torch.datasets

# Load and preprocess your LiDAR dataset
dataset = open3d.ml.torch.datasets.SemanticKITTI(
    root="path/to/dataset", pipeline="voxel",
)

# Create a PointNet model
model = open3d.ml.torch.models.PointNet2SSG(
    num_classes=dataset.num_classes,
    in_channels=dataset.num_features,
)

# Train the model (you would typically have a separate training dataset)
trainer = open3d.ml.torch.pipelines.Trainer(model, dataset)
trainer.train()

# Perform object detection and classification on a new point cloud
new_point_cloud = o3d.io.read_point_cloud("IDD_3D/20220118103808_seq_0/lidar/00000.pcd") # Load or capture your new point cloud
detections = model.inference(new_point_cloud)

# Visualize the results
detections.visualize()


Exception: Open3D was not built with PyTorch support!

In [12]:
import open3d as o3d
import json

# Load the JSON data
with open("IDD_3D/20220118103808_seq_0/label/00000.json", "r") as json_file:
    data = json.load(json_file)

# Initialize an empty list to store bounding box information
bounding_boxes = []

# Iterate through the JSON data and extract bounding box info for cars
for entry in data:
    if entry["obj_type"] == "Car":
        position = entry["psr"]["position"]
        scale = entry["psr"]["scale"]
        rotation = entry["psr"]["rotation"]

        # Create a bounding box using Open3D
        box = o3d.geometry.OrientedBoundingBox(center=o3d.utility.Vector3dVector([position["x"], position["y"], position["z"]]),
                                               R=o3d.utility.Vector3dVector([rotation["x"], rotation["y"], rotation["z"]]),
                                               extent=o3d.utility.Vector3dVector([scale["x"], scale["y"], scale["z"]]))

        bounding_boxes.append(box)

# Load the point cloud data
point_cloud = o3d.io.read_point_cloud("IDD_3D/20220118103808_seq_0/lidar/00000.pcd")

# Create a KDTree for point cloud
kdtree = o3d.geometry.KDTreeFlann(point_cloud)

# Iterate through bounding boxes and count points within each box
num_vehicles = 0
for box in bounding_boxes:
    indices = kdtree.query_ball_point(box.get_center(), max(box.get_extent()))
    num_points_in_box = len(indices)
    if num_points_in_box >= min_threshold:  # Adjust min_threshold as needed
        num_vehicles += 1

print("Number of vehicles:", num_vehicles)


RuntimeError: Unable to cast Python instance to C++ type (compile in debug mode for details)

In [13]:
import open3d as o3d
import json

# Load the JSON data
with open("IDD_3D/20220118103808_seq_0/label/00000.json", "r") as json_file:
    data = json.load(json_file)

# Initialize an empty list to store bounding box information
bounding_boxes = []

# Iterate through the JSON data and extract bounding box info for cars
for entry in data:
    if entry["obj_type"] == "Car":
        position = entry["psr"]["position"]
        scale = entry["psr"]["scale"]
        rotation = entry["psr"]["rotation"]

        # Create a bounding box using Open3D
        center = [position["x"], position["y"], position["z"]]
        rotation_matrix = o3d.geometry.get_rotation_matrix_from_xyz(rotation)
        extent = [scale["x"], scale["y"], scale["z"]]
        box = o3d.geometry.OrientedBoundingBox(center=center, R=rotation_matrix, extent=extent)

        bounding_boxes.append(box)

# Load the point cloud data
point_cloud = o3d.io.read_point_cloud("IDD_3D/20220118103808_seq_0/lidar/00000.pcd")

# Create a KDTree for point cloud
kdtree = o3d.geometry.KDTreeFlann(point_cloud)

# Iterate through bounding boxes and count points within each box
num_vehicles = 0
min_threshold = 10  # Adjust as needed
for box in bounding_boxes:
    indices = kdtree.query_radius(center=box.get_center(), radius=max(box.get_extent()))
    num_points_in_box = len(indices[0])
    if num_points_in_box >= min_threshold:
        num_vehicles += 1

print("Number of vehicles:", num_vehicles)


TypeError: get_rotation_matrix_from_xyz(): incompatible function arguments. The following argument types are supported:
    1. (rotation: numpy.ndarray[numpy.float64[3, 1]]) -> numpy.ndarray[numpy.float64[3, 3]]

Invoked with: {'x': 0, 'y': 0, 'z': -1.7533384818324207}

In [19]:
import open3d as o3d
import json
import numpy as np
from sklearn.neighbors import NearestNeighbors

# Load the JSON data
with open("IDD_3D/20220118103808_seq_0/label/00000.json", "r") as json_file:
    data = json.load(json_file)

# Initialize an empty list to store bounding box information
bounding_boxes = []

# Iterate through the JSON data and extract bounding box info for cars
for entry in data:
    if entry["obj_type"] == "Car":
        position = entry["psr"]["position"]
        scale = entry["psr"]["scale"]
        rotation = entry["psr"]["rotation"]

        # Create a rotation matrix from the dictionary values
        rotation_matrix = np.array([[np.cos(rotation["z"]), -np.sin(rotation["z"]), 0],
                                    [np.sin(rotation["z"]), np.cos(rotation["z"]), 0],
                                    [0, 0, 1]])

        # Create a bounding box using Open3D
        center = [position["x"], position["y"], position["z"]]
        extent = [scale["x"], scale["y"], scale["z"]]
        box = o3d.geometry.OrientedBoundingBox(center=center, R=rotation_matrix, extent=extent)

        bounding_boxes.append(box)

# Load the point cloud data
point_cloud = o3d.io.read_point_cloud("IDD_3D/20220118103808_seq_0/lidar/00000.pcd")

# Extract the point cloud as a NumPy array
point_cloud_data = np.asarray(point_cloud.points)

# Create a NearestNeighbors object for point cloud
kdtree = NearestNeighbors(radius=max(max(box.extent) for box in bounding_boxes))
kdtree.fit(point_cloud_data)

# Iterate through bounding boxes and count points within each box
num_vehicles = 0
min_threshold = 10  # Adjust as needed
for box in bounding_boxes:
    indices = kdtree.radius_neighbors([box.center], max(box.extent))
    num_points_in_box = len(indices[0])
    if num_points_in_box >= min_threshold:
        num_vehicles += 1

print("Number of vehicles:", num_vehicles)


Number of vehicles: 0


In [21]:
!pip install filterpy

                                              0.0/178.0 kB ? eta -:--:--
                                              0.0/178.0 kB ? eta -:--:--
     --                                       10.2/178.0 kB ? eta -:--:--
     --                                       10.2/178.0 kB ? eta -:--:--
     ------                                30.7/178.0 kB 220.2 kB/s eta 0:00:01
     --------                              41.0/178.0 kB 219.4 kB/s eta 0:00:01
     -----------------                     81.9/178.0 kB 353.1 kB/s eta 0:00:01
     ------------------------------------ 178.0/178.0 kB 671.0 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110542 sha256=97d4a6d12c6f493a3f1ac0bf5a1814e2d71e71edea2e8b323c245624bb8b0f1b
  Stored in directory: c:\users\akash\appdata\local\pip\cache\wheels\53\e6\de\a09ea01e923aaf88b9f8c7c44329e857b2c1a31901167e55e6


In [23]:
import cv2
import numpy as np
from filterpy.kalman import KalmanFilter
from collections import defaultdict

# Initialize Kalman Filter for object tracking
def init_kalman_filter():
    kf = KalmanFilter(dim_x=4, dim_z=2)
    kf.F = np.array([[1, 1, 0, 0],
                    [0, 1, 0, 0],
                    [0, 0, 1, 1],
                    [0, 0, 0, 1]])
    kf.H = np.array([[1, 0, 0, 0],
                    [0, 0, 1, 0]])
    kf.x = np.array([0, 0, 0, 0])
    kf.P *= 1
    return kf

# Perform object detection in each camera image
def detect_objects(image, cascade_classifier):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    objects = cascade_classifier.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    return objects

# Initialize Kalman filters for object tracking
kalman_filters = [init_kalman_filter() for _ in range(6)]

# Initialize object tracks
object_tracks = defaultdict(list)

# Initialize Haar Cascade Classifier for vehicle detection
vehicle_cascade = cv2.CascadeClassifier("haarcascade_car.xml")

# Process camera images
for camera_id in range(6):
    for frame_id in range(100):
        image = cv2.imread(f"IDD_3D/20220118103808_seq_0/camera/cam{camera_id}/{frame_id:05d}.png")
        detected_objects = detect_objects(image, vehicle_cascade)
        
        for obj in detected_objects:
            obj_id = len(object_tracks[camera_id])  # Assign a new ID
            kalman_filter = kalman_filters[camera_id]
            
            # Initial state estimation based on detection
            x, y, w, h = obj
            center_x = x + w // 2
            center_y = y + h // 2
            measurement = np.array([center_x, center_y])
            
            kalman_filter.x = np.array([center_x, 0, center_y, 0])
            kalman_filter.P *= 1
            
            kalman_filter.update(measurement)
            object_tracks[camera_id].append({'id': obj_id, 'kalman_filter': kalman_filter})
            
        # Draw object tracks on the image
        for obj in object_tracks[camera_id]:
            x, y = obj['kalman_filter'].x[:2]
            cv2.circle(image, (int(x), int(y)), 3, (0, 255, 0), -1)
        
        cv2.imshow(f"Camera {camera_id}", image)
        cv2.waitKey(1)

# Merge object tracks from all cameras and count vehicles
all_object_tracks = []
for camera_id in range(6):
    all_object_tracks.extend(object_tracks[camera_id])

# Count the number of unique vehicles
vehicle_count = len(set(obj['id'] for obj in all_object_tracks))

print(f"Total number of vehicles: {vehicle_count}")

# Release the OpenCV windows
cv2.destroyAllWindows()


Total number of vehicles: 522


In [34]:
import cv2

# Load the YOLO model for object detection
yolo_net = cv2.dnn.readNet(r"YOLO_files/yolo-voc.weights", "YOLO_files/yolo-voc.cfg")

# Load class labels for vehicle categories
with open("YOLO_files/voc_classes.txt") as f:
    class_labels = f.read().strip().split('\n')

# Function to detect and classify vehicles in an image using YOLO
def detect_and_classify_vehicles_yolo(image):
    # Preprocess the image
    blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    yolo_net.setInput(blob)
    
    # Get the YOLO model's output layer names
    output_layers = yolo_net.getUnconnectedOutLayersNames()
    
    # Run YOLO forward pass
    detections = yolo_net.forward(output_layers)
    
    vehicle_categories = []
    
    # Process YOLO detections
    for detection in detections:
        for obj in detection:
            scores = obj[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:  # Adjust confidence threshold as needed
                label = class_labels[class_id]
                vehicle_categories.append(label)
    
    return vehicle_categories

# Path to the directory containing images (seq_5)
image_dir = "IDD_3D/20220118103808_seq_5/camera"

# Number of frames in seq_5
num_frames = 100

# Initialize a dictionary to store the counts of different vehicle categories
vehicle_counts = {
    'aeroplane': 0,
    'bicycle': 0,
    'bird': 0,
    'boat': 0,
    'bottle': 0,
    'bus': 0,
    'car': 0,
    'cat': 0,
    'chair': 0,
    'cow': 0,
    'diningtable': 0,
    'dog': 0,
    'horse': 0,
    'motorbike': 0,
    'person': 0,
    'pottedplant': 0,
    'sheep': 0,
    'sofa': 0,
    'train': 0,
    'tvmonitor': 0
    # Add more categories as needed
}

# Iterate through the frames (00500.png to 00599.png)
for frame_id in range(500, 600):
    for cam_id in range(6):
        # Construct the file path for the current image
        image_path = f"{image_dir}/cam{cam_id}/{frame_id:05d}.png"

        # Load the image using OpenCV
        image = cv2.imread(image_path)

        # Detect and classify vehicles in the image using YOLO
        vehicle_categories = detect_and_classify_vehicles_yolo(image)

        # Update the counts based on vehicle categories
        for category in vehicle_categories:
            vehicle_counts[category] += 1

# Print the counts of different vehicle categories
for category, count in vehicle_counts.items():
    print(f"{category}: {count}")


aeroplane: 0
bicycle: 0
bird: 0
boat: 3
bottle: 0
bus: 18
car: 448
cat: 0
chair: 2
cow: 0
diningtable: 0
dog: 0
horse: 0
motorbike: 16
person: 149
pottedplant: 1
sheep: 0
sofa: 0
train: 0
tvmonitor: 0


In [26]:
import cv2

# Initialize Haar Cascade Classifier for vehicle detection
vehicle_cascade = cv2.CascadeClassifier("haarcascade_car.xml")

# Function to detect and count vehicles in an image
def detect_and_count_vehicles(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    vehicles = vehicle_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    return len(vehicles)

# Path to the directory containing images in seq_5
image_dir = "IDD_3D/20220118103808_seq_5/camera"

# Number of frames in seq_5
num_frames = 100

# Initialize a dictionary to store the vehicle counts for each frame
vehicle_counts = {}

# Iterate through the frames
for frame_id in range(num_frames):
    total_count = 0  # Initialize the total count for the current frame
    for cam_id in range(6):
        # Construct the file path for the current image
        image_path = f"{image_dir}/cam{cam_id}/{frame_id+500:05d}.png"
        
        # Load the image using OpenCV
        image = cv2.imread(image_path)
        
        # Detect and count vehicles in the image
        num_vehicles = detect_and_count_vehicles(image)
        
        # Add the count for the current camera view to the total count
        total_count += num_vehicles

    # Store the total count for the current frame
    vehicle_counts[f"Frame {frame_id}"] = total_count

# Print the total vehicle counts for each frame in seq_5
for frame, count in vehicle_counts.items():
    print(f"{frame}: Total Vehicles = {count}")


Frame 0: Total Vehicles = 18
Frame 1: Total Vehicles = 21
Frame 2: Total Vehicles = 20
Frame 3: Total Vehicles = 23
Frame 4: Total Vehicles = 17
Frame 5: Total Vehicles = 14
Frame 6: Total Vehicles = 18
Frame 7: Total Vehicles = 18
Frame 8: Total Vehicles = 25
Frame 9: Total Vehicles = 21
Frame 10: Total Vehicles = 18
Frame 11: Total Vehicles = 15
Frame 12: Total Vehicles = 20
Frame 13: Total Vehicles = 22
Frame 14: Total Vehicles = 20
Frame 15: Total Vehicles = 21
Frame 16: Total Vehicles = 21
Frame 17: Total Vehicles = 17
Frame 18: Total Vehicles = 16
Frame 19: Total Vehicles = 21
Frame 20: Total Vehicles = 12
Frame 21: Total Vehicles = 14
Frame 22: Total Vehicles = 18
Frame 23: Total Vehicles = 13
Frame 24: Total Vehicles = 10
Frame 25: Total Vehicles = 13
Frame 26: Total Vehicles = 10
Frame 27: Total Vehicles = 12
Frame 28: Total Vehicles = 11
Frame 29: Total Vehicles = 12
Frame 30: Total Vehicles = 14
Frame 31: Total Vehicles = 16
Frame 32: Total Vehicles = 14
Frame 33: Total Vehi

In [41]:
import cv2

# Initialize Haar Cascade Classifier for vehicle detection
vehicle_cascade = cv2.CascadeClassifier("haarcascade_car.xml")

# Function to detect and count vehicles in an image
def detect_and_count_vehicles(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    vehicles = vehicle_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    # Draw bounding boxes around detected vehicles
    for (x, y, w, h) in vehicles:
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    return len(vehicles), image

# Path to the directory containing images in seq_5
image_dir = "IDD_3D/20220118103808_seq_5/camera"

# Number of frames in seq_5
num_frames = 100

# Initialize a dictionary to store the vehicle counts and images for each frame
vehicle_info = {}

# Iterate through the frames
for frame_id in range(num_frames):
    total_count = 0  # Initialize the total count for the current frame
    frame_images = []  # Initialize a list to store individual camera images
    
    for cam_id in range(6):
        # Construct the file path for the current image
        image_path = f"{image_dir}/cam{cam_id}/{frame_id+500:05d}.png"
        
        # Load the image using OpenCV
        image = cv2.imread(image_path)
        
        # Detect and count vehicles in the image and get the image with bounding boxes
        num_vehicles, image_with_boxes = detect_and_count_vehicles(image)
        
        # Add the count for the current camera view to the total count
        total_count += num_vehicles
        
        # Store the individual camera image with bounding boxes
        frame_images.append(image_with_boxes)
    
    # Store the total count and individual camera images
    vehicle_info[f"Frame {frame_id}"] = (total_count, frame_images)

# Print the total vehicle counts for each frame in seq_5
for frame, (count, frame_images) in vehicle_info.items():
    print(f"{frame}: Total Vehicles = {count}")
    
    # Show the individual camera images with bounding boxes
    for i, cam_image in enumerate(frame_images):
        cv2.imshow(f"{frame} - Camera {i}", cam_image)
    
    cv2.waitKey(0)

# Close OpenCV windows
cv2.destroyAllWindows()


Frame 0: Total Vehicles = 18
Frame 1: Total Vehicles = 21
Frame 2: Total Vehicles = 20
Frame 3: Total Vehicles = 23


KeyboardInterrupt: 

In [30]:
import cv2
import numpy as np

# Load the YOLO model for object detection
yolo_net = cv2.dnn.readNet(r"YOLO_files/yolo-voc.weights", "YOLO_files/yolo-voc.cfg")

# Load class labels for vehicle categories
with open("YOLO_files/voc_classes.txt") as f:
    class_labels = f.read().strip().split('\n')

# Function to detect and classify vehicles in an image using YOLO
def detect_and_classify_vehicles_yolo(image):
    # Preprocess the image
    blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    yolo_net.setInput(blob)
    
    # Get the YOLO model's output layer names
    output_layers = yolo_net.getUnconnectedOutLayersNames()
    
    # Run YOLO forward pass
    detections = yolo_net.forward(output_layers)
    
    vehicle_categories = []
    
    # Process YOLO detections
    for detection in detections:
        for obj in detection:
            scores = obj[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:  # Adjust confidence threshold as needed
                label = class_labels[class_id]
                vehicle_categories.append(label)
    
    return vehicle_categories

# Path to the directory containing images
image_dir = "IDD_3D/20220118103808_seq_0/camera"

# Number of frames
num_frames = 100

# Initialize a dictionary to store the counts of different vehicle categories
vehicle_counts = {
    'aeroplane': 0,
    'bicycle': 0,
    'bird': 0,
    'boat': 0,
    'bottle': 0,
    'bus': 0,
    'car': 0,
    'cat': 0,
    'chair': 0,
    'cow': 0,
    'diningtable': 0,
    'dog': 0,
    'horse': 0,
    'motorbike': 0,
    'person': 0,
    'pottedplant': 0,
    'sheep': 0,
    'sofa': 0,
    'train': 0,
    'tvmonitor': 0
    # Add more categories as needed
}

# Iterate through all frames
for frame_id in range(num_frames):
    for cam_id in range(6):
        # Construct the file path for the current image
        image_path = f"{image_dir}/cam{cam_id}/{frame_id:05d}.png"

        # Load the image using OpenCV
        image = cv2.imread(image_path)

        # Detect and classify vehicles in the image using YOLO
        vehicle_categories = detect_and_classify_vehicles_yolo(image)

        # Update the counts based on vehicle categories
        for category in vehicle_categories:
            vehicle_counts[category] += 1

# Print the counts of different vehicle categories with counts greater than 0
for category, count in vehicle_counts.items():
    if count > 1:
        print(f"{category}: {count}")


bicycle: 2
car: 309
motorbike: 68
person: 126
pottedplant: 1
train: 17


In [38]:
import cv2

# Load the YOLO model for object detection
yolo_net = cv2.dnn.readNet(r"YOLO_files/yolo-voc.weights", "YOLO_files/yolo-voc.cfg")

# Load class labels for vehicle categories
with open("YOLO_files/voc_classes.txt") as f:
    class_labels = f.read().strip().split('\n')

# Function to detect and classify vehicles in an image using YOLO
def detect_and_classify_vehicles_yolo(image):
    # Preprocess the image
    blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    yolo_net.setInput(blob)
    
    # Get the YOLO model's output layer names
    output_layers = yolo_net.getUnconnectedOutLayersNames()
    
    # Run YOLO forward pass
    detections = yolo_net.forward(output_layers)
    
    vehicle_categories = []
    
    # Process YOLO detections
    for detection in detections:
        for obj in detection:
            scores = obj[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:  # Adjust confidence threshold as needed
                label = class_labels[class_id]
                vehicle_categories.append(label)
    
    return vehicle_categories

# Path to the directory containing images (seq_0)
image_dir = "IDD_3D/20220118103808_seq_1/camera"

# Initialize a dictionary to store the counts of different vehicle categories
vehicle_counts = {
    'bicycle': 0,
    'bus': 0,
    'car': 0,
    'motorbike': 0,
    'person': 0,
    'train': 0
    # Add more categories as needed
}

# Iterate through cam0 to cam5 for "00000.png"
for cam_id in range(6):
    # Construct the file path for the current image
    image_path = f"{image_dir}/cam{cam_id}/00100.png"

    # Load the image using OpenCV
    image = cv2.imread(image_path)

    # Detect and classify vehicles in the image using YOLO
    vehicle_categories = detect_and_classify_vehicles_yolo(image)

    # Update the counts based on vehicle categories
    for category in vehicle_categories:
        vehicle_counts[category] += 1

# Print the counts of different vehicle categories with counts greater than 0
for category, count in vehicle_counts.items():
    if count > 0:
        print(f"{category}: {count}")


car: 2
motorbike: 1
person: 5


In [43]:
import cv2

# Load the YOLO model for object detection
yolo_net = cv2.dnn.readNet(r"YOLO_files/yolo-voc.weights", "YOLO_files/yolo-voc.cfg")

# Load class labels for vehicle categories
with open("YOLO_files/voc_classes.txt") as f:
    class_labels = f.read().strip().split('\n')

# Function to detect and classify vehicles in an image using YOLO
def detect_and_classify_vehicles_yolo(image):
    # Preprocess the image
    blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    yolo_net.setInput(blob)
    
    # Get the YOLO model's output layer names
    output_layers = yolo_net.getUnconnectedOutLayersNames()
    
    # Run YOLO forward pass
    detections = yolo_net.forward(output_layers)
    
    vehicle_categories = []
    vehicle_bboxes = []  # List to store bounding boxes
    
    # Process YOLO detections
    for detection in detections:
        for obj in detection:
            scores = obj[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:  # Adjust confidence threshold as needed
                label = class_labels[class_id]
                vehicle_categories.append(label)
                
                # Get bounding box coordinates
                center_x, center_y, width, height = map(int, obj[0:4] * [image.shape[1], image.shape[0], image.shape[1], image.shape[0]])
                x = center_x - width // 2
                y = center_y - height // 2
                
                vehicle_bboxes.append((x, y, width, height))
    
    # Draw bounding boxes on the image
    for bbox in vehicle_bboxes:
        x, y, width, height = bbox
        cv2.rectangle(image, (x, y), (x + width, y + height), (0, 255, 0), 2)
    
    return vehicle_categories, image

# Path to the directory containing images (seq_0)
image_dir = "IDD_3D/20220118103808_seq_1/camera"

# Initialize a dictionary to store the counts of different vehicle categories
vehicle_counts = {
    'bicycle': 0,
    'bus': 0,
    'car': 0,
    'motorbike': 0,
    'person': 0,
    'train': 0
    # Add more categories as needed
}

# Iterate through cam0 to cam5 for "00100.png"
for cam_id in range(6):
    # Construct the file path for the current image
    image_path = f"{image_dir}/cam{cam_id}/00100.png"

    # Load the image using OpenCV
    image = cv2.imread(image_path)

    # Detect and classify vehicles in the image using YOLO
    vehicle_categories, image_with_boxes = detect_and_classify_vehicles_yolo(image)

    # Update the counts based on vehicle categories
    for category in vehicle_categories:
        vehicle_counts[category] += 1

    # Display the image with bounding boxes
    cv2.imshow(f"Camera {cam_id}", image_with_boxes)

# Print the counts of different vehicle categories with counts greater than 0
for category, count in vehicle_counts.items():
    if count > 0:
        print(f"{category}: {count}")

# Wait for a key press and then close OpenCV windows
cv2.waitKey(0)
cv2.destroyAllWindows()


car: 2
motorbike: 1
person: 5
